Imports

In [67]:
from src.smallcnn import SmallCNN
from src.preprocessed_dataset import PrecomputedFMADataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from src.audio_processing import audio_to_mel_tensor
import os
import librosa
import numpy as np
import pandas as pd
import ast
from pathlib import Path
from src.rnn import AudioRNN


Data préparation

In [ ]:
root = "fma_small"
file_paths = []
labels = []

for dirpath, dirnames, filenames in os.walk(root):
    for filename in filenames:
        if filename.endswith(".mp3"):
            full_path = os.path.join(dirpath, filename)

            label = os.path.basename(dirpath)

            file_paths.append(full_path)
            labels.append(int(label))

valid_rows = []

tracks_df = pd.read_csv("fma_metadata/tracks.csv", header=[0,1], index_col=0)

genres_raw = tracks_df[("track", "genres")]
track_genre_id_map = {}

for tid, string_list in genres_raw.items():
    try:
        genre_ids = ast.literal_eval(string_list)
        if len(genre_ids) > 0:
            track_genre_id_map[tid] = genre_ids[0]
    except:
        pass

track_genre_ids = []

for path in file_paths:
    filename = os.path.basename(path)
    track_id = int(filename.replace(".mp3", ""))

    if track_id in track_genre_id_map:
        track_genre_ids.append(track_genre_id_map[track_id])
    else:
        track_genre_ids.append(None)  

for path, gid in zip(file_paths, track_genre_ids):
    if gid is None:
        continue

    mel, sr = audio_to_mel_tensor(path)
    if mel is None:
        print(f"On ignore le fichier corrompu : {path}")
        continue

    valid_rows.append((path, gid))

print(f"Nombre de fichiers valides : {len(valid_rows)}")

# On reconstruit les listes propres
valid_paths = [r[0] for r in valid_rows]
valid_genre_ids = [r[1] for r in valid_rows]

/Users/remi-moustamsik/Documents/projet_lock/src/audio_processing.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=sr, mono=True)
[src/libmpg123/parse.c:do_readahead():1083] warning: Cannot read next header, a one-frame stream? Duh...
/Users/remi-moustamsik/Documents/projet_lock/lock-env/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Python(18148) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Erreur de chargement fma_small/133/133297.mp3: 
On ignore le fichier corrompu : fma_small/133/133297.mp3


[src/libmpg123/layer3.c:INT123_do_layer3():1776] error: part2_3_length (3360) too large for available bit count (3240)
[src/libmpg123/layer3.c:INT123_do_layer3():1776] error: part2_3_length (3328) too large for available bit count (3240)
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 33361.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1349] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
/Users/remi-moustamsik/Documents/projet_lock/src/audio_processing.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=sr, mono=True)
/Users/remi-moustamsik/Documents/projet_lock/lock-env/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype

Erreur de chargement fma_small/099/099134.mp3: 
On ignore le fichier corrompu : fma_small/099/099134.mp3


/Users/remi-moustamsik/Documents/projet_lock/src/audio_processing.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=sr, mono=True)
[src/libmpg123/parse.c:do_readahead():1083] warning: Cannot read next header, a one-frame stream? Duh...
/Users/remi-moustamsik/Documents/projet_lock/lock-env/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Python(19011) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Erreur de chargement fma_small/108/108925.mp3: 
On ignore le fichier corrompu : fma_small/108/108925.mp3


[src/libmpg123/layer3.c:INT123_do_layer3():1844] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1804] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1804] error: dequantization failed!


Nombre de fichiers valides : 7997


Mapping

In [33]:
unique_genre_ids = sorted(set(valid_genre_ids))
genre_id_to_idx = {gid: i for i, gid in enumerate(unique_genre_ids)}
idx_to_genre_id = {i: gid for gid, i in genre_id_to_idx.items()}
num_classes = len(unique_genre_ids)
print("Nombre de classes après filtrage:", num_classes)

Nombre de classes après filtrage: 103


Test

In [ ]:
model.train()
optimizer.zero_grad()
inputs, labels = next(iter(train_loader))
inputs = inputs.to(device).float(); labels = labels.to(device)
logits, _ = model(inputs)
loss = criterion(logits, labels)
loss.backward()
print("some grad stats:", model.conv_block1[0].weight.grad.norm().item())

import pandas as pd
md = pd.read_csv("precomputed_mels/metadata.csv")
print("num_classes var:", num_classes)
print("label_idx max:", md["label_idx"].max(), "n_unique:", md["label_idx"].nunique())

some grad stats: 113.45136260986328
num_classes var: 103
label_idx max: 102 n_unique: 103


Stockage features précalculées.
Cela permet de gagner du temps en évitant de process les fichierrs audio à chque époque

In [ ]:
features_root = Path("precomputed_mels")
features_root.mkdir(parents=True, exist_ok=True)

metadata_rows = []

for path, gid in zip(valid_paths, valid_genre_ids):
    # On calcule le mel
    mel, sr = audio_to_mel_tensor(path)
    if mel is None:
        continue

    mel = mel.float()

    filename = os.path.basename(path)
    track_id_str = filename.replace(".mp3", "")

    out_path = features_root / f"{track_id_str}.pt"

    # Sauvegarde tensor sur disque
    torch.save(mel, out_path)

    label_idx = genre_id_to_idx[gid]

    metadata_rows.append({
        "track_id": int(track_id_str),
        "audio_path": path,
        "feature_path": str(out_path),
        "genre_id": gid,
        "label_idx": label_idx,
    })

[src/libmpg123/layer3.c:INT123_do_layer3():1776] error: part2_3_length (3360) too large for available bit count (3240)
[src/libmpg123/layer3.c:INT123_do_layer3():1776] error: part2_3_length (3328) too large for available bit count (3240)
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 33361.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1349] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
/Users/remi-moustamsik/Documents/projet_lock/src/audio_processing.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=sr, mono=True)
/Users/remi-moustamsik/Documents/projet_lock/lock-env/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype

Sauvegarder les metadata

In [11]:
metadata_df = pd.DataFrame(metadata_rows)
metadata_df.to_csv(features_root / "metadata.csv", index=False)

print("✅ Pré-calcul terminé.")
print(f"Features sauvegardées dans : {features_root}")
print(f"Metadata : {features_root / 'metadata.csv'}")

✅ Pré-calcul terminé.
Features sauvegardées dans : precomputed_mels
Metadata : precomputed_mels/metadata.csv


In [14]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✓ Entraînement sur GPU Apple (MPS)")
else:
    device = torch.device("cpu")
    print("⚠️ Entraînement sur CPU")

✓ Entraînement sur GPU Apple (MPS)


In [ ]:
metadata_csv = "precomputed_mels/metadata.csv"

dataset = PrecomputedFMADataset(metadata_csv, max_len=3000)

train_size = int(0.8 * len(dataset))
test_size  = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=16, shuffle=False)


In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in dataloader:
        inputs = inputs.to(device).float()
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        logits, _ = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device).float()
            labels = labels.to(device)
            
            logits, probs = model(inputs)
            loss = criterion(logits, labels)
            
            running_loss += loss.item() * inputs.size(0)
            
            _, preds = torch.max(probs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = correct / total
    return epoch_loss, accuracy



In [62]:
model = SmallCNN(n_mels=128, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.to(device)

SmallCNN(
  (conv_block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
  )
  (conv_block3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
  )
  (conv_block4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
  

In [64]:
NUM_EPOCHS = 20


metadata_csv = "precomputed_mels/metadata.csv"

dataset = PrecomputedFMADataset(metadata_csv, max_len=3000)
dataset_size = len(dataset)

train_size = int(0.8 * len(dataset))
test_size  = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=2, shuffle=False)


for inputs, labels in test_loader:
    print("inputs shape, dtype, device:", inputs.shape, inputs.dtype)
    print("labels sample, min/max:", labels[:10], labels.min().item(), labels.max().item())
    print("feature stats:", inputs.min().item(), inputs.mean().item(), inputs.max().item())
    break

import pandas as pd
md = pd.read_csv("precomputed_mels/metadata.csv")
print(md.columns)
print("unique genre_id:", md["genre_id"].nunique(), "unique label_idx:", md["label_idx"].nunique())
print("label_idx unique sample:", sorted(md["label_idx"].unique())[:10])

for inputs, labels in train_loader:
    inputs = inputs.to(device).float()
    labels = labels.to(device)
    
    print(f"Input shape: {inputs.shape}, dtype: {inputs.dtype}, device: {inputs.device}")
    print(f"Labels shape: {labels.shape}, dtype: {labels.dtype}, device: {labels.device}")
    
    logits, probs = model(inputs)
    print(f"Logits shape: {logits.shape}")
    print(f"Probs shape: {probs.shape}")
    break

for epoch in range(NUM_EPOCHS):
    # ---- TRAIN ----
    train_loss = train_one_epoch(
        model=model,
        dataloader=train_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device
    )
    
    # ---- TEST ----
    test_loss, test_acc = evaluate(
        model=model,
        dataloader=test_loader,
        criterion=criterion,
        device=device
    )

    if torch.backends.mps.is_available():
        try:
            torch.mps.empty_cache()
        except Exception:
            pass

    _, preds = torch.max(probs, dim=1)
    print(f"Sample preds: {preds[:5]}, Sample labels: {labels[:5]}")
    print(f"Unique preds: {torch.unique(preds)}, Unique labels: {torch.unique(labels)}")

    print(
        f"Epoch {epoch+1}/{NUM_EPOCHS} "
        f"- train loss: {train_loss:.4f} | test loss: {test_loss:.4f} | test acc: {test_acc*100:.2f}%"
    )


inputs shape, dtype, device: torch.Size([2, 1, 128, 3000]) torch.float32
labels sample, min/max: tensor([ 6, 81]) 6 81
feature stats: -1.9761807918548584 1.017252593271678e-08 2.61867356300354
Index(['track_id', 'audio_path', 'feature_path', 'genre_id', 'label_idx'], dtype='object')
unique genre_id: 103 unique label_idx: 103
label_idx unique sample: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)]
Input shape: torch.Size([2, 1, 128, 3000]), dtype: torch.float32, device: mps:0
Labels shape: torch.Size([2]), dtype: torch.int64, device: mps:0
Logits shape: torch.Size([2, 103])
Probs shape: torch.Size([2, 103])
Sample preds: tensor([7, 8], device='mps:0'), Sample labels: tensor([6, 8], device='mps:0')
Unique preds: tensor([7, 8], device='mps:0'), Unique labels: tensor([6, 8], device='mps:0')
Epoch 1/20 - train loss: 3.2040 | test loss: 3.0253 | test acc: 25.44%
Sample preds: tensor([7, 8], device='mps:0'), Sa

In [65]:
torch.mps.empty_cache()


Code RNN

In [68]:
model = AudioRNN(
    n_mels=128,
    hidden_size=128,
    num_layers=2,
    bidirectional=True,
    embedding_dim=128,
    num_classes=None,
).to(device)

In [69]:
import torch
from torch.utils.data import DataLoader, random_split
import pandas as pd

NUM_EPOCHS = 20

metadata_csv = "precomputed_mels/metadata.csv"

dataset = PrecomputedFMADataset(metadata_csv, max_len=3000)
dataset_size = len(dataset)

train_size = int(0.8 * len(dataset))
test_size  = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=2, shuffle=False)

# device MPS ou CPU
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print("Device:", device)

md = pd.read_csv(metadata_csv)
num_classes = md["label_idx"].nunique()
print("unique genre_id:", md["genre_id"].nunique(), "unique label_idx:", num_classes)
print("label_idx unique sample:", sorted(md["label_idx"].unique())[:10])

# ---- instancier le RNN ----
model = AudioRNN(
    n_mels=128,
    hidden_size=128,
    num_layers=2,
    bidirectional=True,
    embedding_dim=128,
    num_classes=num_classes,
    dropout=0.3,
).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


Device: mps
unique genre_id: 103 unique label_idx: 103
label_idx unique sample: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)]


Tests

In [70]:
for inputs, labels in train_loader:
    inputs = inputs.to(device).float()
    labels = labels.to(device)

    print(f"Input shape: {inputs.shape}, dtype: {inputs.dtype}, device: {inputs.device}")
    print(f"Labels shape: {labels.shape}, dtype: {labels.dtype}, device: {labels.device}")

    logits, probs, emb = model(inputs)
    print(f"Logits shape: {logits.shape}")
    print(f"Probs shape: {probs.shape}")
    print(f"Embeddings shape: {emb.shape}")
    break


Input shape: torch.Size([2, 1, 128, 3000]), dtype: torch.float32, device: mps:0
Labels shape: torch.Size([2]), dtype: torch.int64, device: mps:0
Logits shape: torch.Size([2, 103])
Probs shape: torch.Size([2, 103])
Embeddings shape: torch.Size([2, 128])


Fonctions d'entrainement et d'évaluation

In [71]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_correct = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device).float()
        labels = labels.to(device)

        optimizer.zero_grad()

        logits, probs, emb = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        preds = logits.argmax(dim=1)
        running_correct += (preds == labels).sum().item()

        del inputs, labels, logits, probs, emb, loss
        if torch.backends.mps.is_available():
            torch.mps.empty_cache()

    avg_loss = running_loss / len(dataloader.dataset)
    acc = running_correct / len(dataloader.dataset)
    return avg_loss, acc


@torch.no_grad()
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_correct = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device).float()
        labels = labels.to(device)

        logits, probs, emb = model(inputs)
        loss = criterion(logits, labels)

        running_loss += loss.item() * inputs.size(0)
        preds = logits.argmax(dim=1)
        running_correct += (preds == labels).sum().item()

    avg_loss = running_loss / len(dataloader.dataset)
    acc = running_correct / len(dataloader.dataset)
    return avg_loss, acc


Boucle d'entrainement

In [ ]:
for epoch in range(NUM_EPOCHS):
    # ---- TRAIN ----
    train_loss, train_acc = train_one_epoch(
        model=model,
        dataloader=train_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device
    )
    
    # ---- TEST ----
    test_loss, test_acc = evaluate(
        model=model,
        dataloader=test_loader,
        criterion=criterion,
        device=device
    )

    if torch.backends.mps.is_available():
        try:
            torch.mps.empty_cache()
        except Exception:
            pass

    print(
        f"Epoch {epoch+1}/{NUM_EPOCHS} "
        f"- train loss: {train_loss:.4f} | train acc: {train_acc*100:.2f}% "
        f"| test loss: {test_loss:.4f} | test acc: {test_acc*100:.2f}%"
    )
